What we need is a small parcel to work on, with a nice contourplot with critical points and minima found after initiating local method and then a 3d plot. 

In [1]:
using Pkg
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames

  Activating project at `~/Globtim.jl`


In [15]:
# Constants and Parameters
const n, a, b = 2, 1, 4
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = tref # Objective function

tref (generic function with 1 method)

One may assume that when we have access to exact evaluations, we would want to have a small $L^2$-norm tolerance `tol_l2 = 5e-4` and high probability of computing an accurate discrete $L^2$-norm `alpha= 1/10`.

We need to also return the number of samples used to generate the sample set. It is annoying that the error goes up while the degree has increased.

In [5]:
rand_center = [2*rand()-1, 2*rand()-1]*.75; # Random center

In [16]:
d = 22 # Initial Degree 
SMPL = 80 # Number of samples
TR = test_input(f, 
                dim = n,
                center=rand_center,
                GN=SMPL, 
                sample_range=scale_factor
                )
pol_cheb = Constructor(TR, d, GN=SMPL, basis=:chebyshev)
pol_lege = Constructor(TR, d, GN=SMPL, basis=:legendre);

@polyvar(x[1:n]); # Define polynomial ring 

current L2-norm: 0.0011989155034879582
Number of samples: 80
current L2-norm: 0.001257967821238072
Number of samples: 80


Solve the system of partial derivatives using `Msolve`. 

In [ ]:
msolve_polynomial_system(pol_cheb, x)
df_cheb = msolve_parser("outputs.ms", f, n, TR)

Input file already exists: inputs.ms
Output file already exists: outputs.ms

---------------- TIMINGS ----------------
overall(elapsed)        0.00 sec
overall(cpu)            0.00 sec
select                  0.00 sec  13.8%
symbolic prep.          0.00 sec  48.1%
update                  0.00 sec   2.0%
convert                 0.00 sec  15.2%
linear algebra          0.00 sec  13.9%
reduce gb               0.00 sec   0.0%
-----------------------------------------

---------- COMPUTATIONAL DATA -----------
size of basis                    22
#terms in basis                8204
#pairs reduced                   22
#GM criterion                   231
#redundant elements               1
#rows reduced                    66
#zero reductions                  1
max. matrix data                 45 x 485 (78.080%)
max. symbolic hash table size  2^11
max. basis hash table size     2^16
-----------------------------------------


---------------- TIMINGS ----------------
overall(elapsed)        0.01


--------------- INPUT DATA ---------------
#variables                       2
#equations                       2
#invalid equations               0
field characteristic             0
homogeneous input?               0
signature-based computation      0
monomial order                 DRL
basis hash table resetting     OFF
linear algebra option            2
initial hash table size     131072 (2^17)
max pair selection             ALL
reduce gb                        1
#threads                         1
info level                       1
generate pbm files               0
------------------------------------------


{8}{16}{32}{64}{128}
<Step:2/0.00/0.02>
<Step:4/0.01/0.03>{256}

[0][1]Real root isolation starts at precision 288
Number of real roots: 185
Starts real root extraction.
<1426><1416><1413><1421><1442><1420>{2.70%}<1443><1426><1420><1355><1425><1430>{5.95%}<1438><-<1428><1418><1416><1443>

In [14]:
plot_polyapprox(pol_cheb, TR, df_cheb)

In [25]:
using GLMakie
function plot_polyapprox_rotate(pol::ApproxPoly, TR::test_input, df::DataFrame)
    # Extract coordinates and function values
    coords = pol.scale_factor * pol.grid .+ TR.center'
    z_coords = pol.z

    if size(coords)[2] == 2  # Plot if the dimensions are 2
        fig = Figure(size=(800, 600))
        ax = Axis3(fig[1, 1], title="Trefethen Function",
            xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

        # Create a regular grid for surface plotting
        x_unique = sort(unique(coords[:, 1]))
        y_unique = sort(unique(coords[:, 2]))

        # Initialize the Z grid with NaN
        Z = fill(NaN, (length(y_unique), length(x_unique)))

        # Fill the Z grid by matching coordinates
        for (idx, (x, y, z)) in enumerate(zip(coords[:, 1], coords[:, 2], z_coords))
            i = findlast(≈(y), y_unique)
            j = findlast(≈(x), x_unique)
            if !isnothing(i) && !isnothing(j)
                Z[j, i] = z
            end
        end

        # Create surface plot
        surface!(ax, x_unique, y_unique, Z,
            colormap=:viridis,
            transparency=true,
            alpha=0.8)

        # Plot the critical points
        scatter!(ax, df.x1, df.x2,
            df.z,
            markersize=10,
            color=:orange,
            label="Chebyshev approximant critical points")

        # Record rotating animation
        record(fig, "polyapprox_rotation.mp4", 1:240; framerate=30) do frame
            ax.azimuth[] = 1.7pi + 0.4 * sin(2pi * frame / 240)
            ax.elevation[] = pi / 4 + 0.3 * cos(2pi * frame / 240)
        end

        display(fig)
        return fig
    end
end

plot_polyapprox_rotate (generic function with 1 method)

In [26]:
plot_polyapprox_rotate(pol_cheb, TR, df_cheb)